# Seach optimal augmentation intensity by $\mu$ (minimal mutual information)

In [2]:
import os
os.chdir("/home1/quanquan/code/landmark/code/tproj")
assert os.getcwd() == "/home1/quanquan/code/landmark/code/tproj"

In [3]:
import cv2
from sklearn.metrics import mutual_info_score
from scipy.stats import entropy
# from scipy.misc import imread
import numpy as np
import cv2
from skimage.exposure import histogram
from scipy.stats import entropy
import matplotlib
from tutils import tfilename
from PIL import Image
import torchvision.transforms.functional as F
from scipy.stats import norm
matplotlib.use('Agg')  # Must be before importing matplotlib.pyplot or pylab!
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import seaborn as sns
from skimage.filters.rank import entropy
# from scipy.stats import entropy
from skimage.morphology import disk
from tqdm import tqdm
from torchvision.utils import save_image
import torch

mi_mean = [1.1777022788]
mi_std = [0.432]

miself_mean = [2.51138927,3.18064627,3.1936682,2.11334156,3.26182041,2.72902131
            ,2.16312098,1.94328915,2.01532239,3.26112403,3.20813622,3.16552408
            ,3.12697808,2.64613941,3.25124197,1.97482131,3.206975,3.22484813
            ,2.66378744]
miself_std = [0.46076691,0.30574453,0.2845213,0.48100281,0.24713011,0.75816679
,0.75060892,0.65067037,0.6793318, 0.23704422,0.36165119,0.38236706
,0.30267382,0.60297583,0.24602729,0.60459519,0.21163927,0.24409171
,0.47145102]

entr_mean = [4.32922599,4.94955578,4.9402223,4.13369406,5.02937445,4.46131056
            ,4.12397278,4.07084514,4.08404715,4.8864486, 4.87044201,4.79871701
            ,4.7067016, 4.29701387,4.94452249,4.05091702,5.03718015,5.01795204
            ,4.53197619]
entr_std = [0.32104817,0.36015503,0.25001072,0.34826474,0.22153489,0.53958994
,0.44612867,0.39377246,0.4095514, 0.27827981,0.33560316,0.348146
,0.33046936,0.41912717,0.26862784,0.38550231,0.17205685,0.20877866
,0.32103766]

mre = [1.399, 1.5526,1.7562,2.9915 , 1.7145, 1.9027,
       1.8260, 1.3118, 1.1585, 4.4261, 2.2153, 2.8174,
       1.5348, 1.9849, 1.3075, 1.8713,1.5415, 3.7358,
       2.3073]
mre2 =[1.6767,1.7246,2.2157,2.5067,1.9819,2.2091,
       1.7173,1.3327,1.6392,2.2740,1.6451,1.7791,
       1.5442,1.7209,1.8208,2.9706,1.8948,1.9768,
       2.1830]

def get_fea(patch):
    fea = np.zeros((256,))
    hist, idx = histogram(patch, nbins=256)
    for hi, idi in zip(hist, idx):
        # print(hi, idi, i, j)
        fea[idi] = hi
    return fea

def _tmp_fn2(landmark_id, alpha=1):
    """ augment and crop patch (different from _tmp_fn() ) """
    from datasets.ceph.ceph_ssl import Test_Cephalometric
    pth = '/home1/quanquan/datasets/Cephalometric/'
    testset = Test_Cephalometric(pth, mode="Train")
    # Process: lm 0, 3, 7
    lms = testset.ref_landmarks(114)   

    dirname = '/home1/quanquan/datasets/Cephalometric/RawImage/TrainingData/'

    params_list = []
    for i in range(150):
        i += 1
        # i = 115
        # landmark_id = 3
        
        im_name = f'{i:03d}.bmp'
        im = cv2.imread(dirname + im_name, cv2.IMREAD_GRAYSCALE)
        im = cv2.resize(im, (384, 384))

        lm = lms[landmark_id]  # [168, 163]
        print()
        ps_half = 32
        patch = im[lm[1] - ps_half:lm[1] + ps_half, lm[0] - ps_half:lm[0] + ps_half]
        # save image patch
        # save_image(torch.Tensor(patch) / 255, f"paper_mi/saved_patches/im{i:03d}_lm{landmark_id}_patch.png")
        # continue
        fea1 = get_fea(patch)
        im = Image.fromarray(im)

        # print("=============================")
        cj_brightness = 1.
        cj_contrast = 1.
        params1 = [0, 0]
        for i in range(100):
            im_aug = F.adjust_brightness(im, cj_brightness)
            im_aug = F.adjust_contrast(im_aug, cj_contrast)
            im_aug = np.array(im_aug)
            # cv2.imwrite("patch2.jpg", patch_aug)
            patch_aug = im_aug[lm[1] - ps_half:lm[1] + ps_half, lm[0] - ps_half:lm[0] + ps_half]
            fea2 = get_fea(patch_aug)
            # mi0 = mutual_info_score(fea1.copy(), fea1.copy())
            mi = mutual_info_score(fea1, fea2)
            print(f"entr: {entr_mean[landmark_id]}, br: {cj_brightness}, ct: {cj_contrast}, mi:", mi, mi_mean[landmark_id] , mi_std[landmark_id]) # mi0
            cj_brightness += 0.03
            cj_contrast += 0.04
            if mi < mi_mean[landmark_id] - mi_std[landmark_id]:  # - mi_std[landmark_id]:
                break
            params1 = [cj_brightness, cj_contrast]
        print("=============================")

        cj_brightness = 1
        cj_contrast = 1
        params2 = [0, 0]
        for i in range(100):
            im_aug = F.adjust_brightness(im, cj_brightness)
            im_aug = F.adjust_contrast(im_aug, cj_contrast)
            im_aug = np.array(im_aug)
            # cv2.imwrite("patch2.jpg", patch_aug)
            patch_aug = im_aug[lm[1] - ps_half:lm[1] + ps_half, lm[0] - ps_half:lm[0] + ps_half]
            fea2 = get_fea(patch_aug)

            # mi0 = mutual_info_score(fea1.copy(), fea1.copy())
            mi = mutual_info_score(fea1, fea2)
            print(f"entr: {entr_mean[landmark_id]}, br: {cj_brightness}, ct: {cj_contrast}, mi:", mi, mi_mean[landmark_id] , mi_std[landmark_id])  # mi0
            cj_brightness -= 0.03
            cj_contrast -= 0.04
            if mi < mi_mean[landmark_id] - mi_std[landmark_id]:
                break
            params2 = [cj_brightness, cj_contrast]
        # print("=============================")
        params_list.append(params1 + params2)
        # import ipdb; ipdb.set_trace()

    params_list = np.array(params_list)
    mean_values = params_list.mean(axis=0)
    return mean_values

dd = _tmp_fn2(0,1)
print(dd)

The sizes are set as  [384, 384]
Initializing Datasets: (split:'Train') (len:(150)) 

entr: 4.32922599, br: 1.0, ct: 1.0, mi: 2.649186257590599 1.1777022788 0.432
entr: 4.32922599, br: 1.03, ct: 1.04, mi: 1.9900890012281494 1.1777022788 0.432
entr: 4.32922599, br: 1.06, ct: 1.08, mi: 1.7496131963906314 1.1777022788 0.432
entr: 4.32922599, br: 1.09, ct: 1.12, mi: 1.6199276127880502 1.1777022788 0.432
entr: 4.32922599, br: 1.12, ct: 1.1600000000000001, mi: 1.474142509531858 1.1777022788 0.432
entr: 4.32922599, br: 1.1500000000000001, ct: 1.2000000000000002, mi: 1.3741691410677448 1.1777022788 0.432
entr: 4.32922599, br: 1.1800000000000002, ct: 1.2400000000000002, mi: 1.2705412883483704 1.1777022788 0.432
entr: 4.32922599, br: 1.2100000000000002, ct: 1.2800000000000002, mi: 1.1611250096811399 1.1777022788 0.432
entr: 4.32922599, br: 1.2400000000000002, ct: 1.3200000000000003, mi: 1.0251026931317586 1.1777022788 0.432
entr: 4.32922599, br: 1.2700000000000002, ct: 1.3600000000000003, mi: 0.